In [3]:
import re
import sys
import json
import tarfile
import urllib3
import configparser

import tensorflow_hub as hub
import tensorflow.compat.v1 as tf

from tqdm import tqdm
from elasticsearch import Elasticsearch

In [4]:
urllib3.disable_warnings()

config = configparser.ConfigParser()
config.read('../server.ini');

In [ ]:
es = Elasticsearch(
    config['ELASTIC']['path'],
    verify_certs=False
)

In [6]:
es.info()

{'name': '1bb166804bb3',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'm81m9H08RKigCdwIje8C6w',
 'version': {'number': '8.11.0',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'd9ec3fa628c7b0ba3d25692e277ba26814820b20',
  'build_date': '2023-11-04T10:04:57.184859352Z',
  'build_snapshot': False,
  'lucene_version': '9.8.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [12]:
specs = []
specs_names = []
specs_versions = []
specs_categories = []

tar = tarfile.open('../data/data.tar.gz', 'r:gz')
nl_tags = r'(?:\'|\")(?:description|name|title|summary)(?:\'|\"):\s(?:\'|\")([^\'\"]+)(?:\'|\")'

for spec in tqdm(tar.getmembers(), file=sys.stdout):
    file = tar.extractfile(spec)
    
    if file is not None:
        try:
            contents = json.load(file)
            
            specs_names.append(contents['info']['title'])
            specs_versions.append(contents['info']['version'] if not contents['info']['version'].startswith('v') else contents['info']['version'][1:])
            specs.append(' '.join(re.findall(nl_tags, str(contents))))
            
            try:
                specs_categories.append(contents['info']['x-apisguru-categories'][0])
            except KeyError:
                specs_categories.append('none')
        except json.JSONDecodeError:
            continue

100%|██████████| 3996/3996 [00:13<00:00, 304.24it/s]


In [ ]:
print('Downloading pre-trained embeddings from tensorflow hub...')
tf.disable_eager_execution()
embed = hub.Module('../out/models/universal-sentence-encoder')
text_ph = tf.placeholder(tf.string)
embeddings = embed(text_ph)
print('Done.')

In [ ]:
print('Creating tensorflow session...')
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
session = tf.Session(config=config)
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())
print('Done.')

In [16]:
es.indices.create(
    index='oa-specifications',
    body={
        'mappings': {
            'properties': {
                "document_vector": {
                    "type": "dense_vector",
                    "dims": 512
                },              
                "name" : {
                    "type" : "keyword"
                },
                "version" : {
                    "type" : "keyword"
                },
                "category" : {
                    "type" : "keyword"
                }
            }
        }
    }
)

for ind in tqdm(range(len(specs))):
    es.index(
        index='oa-specifications',
        body={
            'name': specs_names[ind],
            'version': specs_versions[ind],
            'category': specs_categories[ind],
            'document_vector': session.run(embeddings, feed_dict={text_ph: [specs[ind]]}).tolist()[0]
        }
    )

100%|██████████| 3992/3992 [00:38<00:00, 104.07it/s]


In [17]:
query = 'driving license registration service'

knn_query = {
    'knn': {
        'field': 'document_vector',
        'query_vector': session.run(embeddings, feed_dict={text_ph: [query]}).tolist()[0],
        'k': 5,
        'num_candidates': 3992
    }
}

results = es.search(index='oa-specifications', body=knn_query, _source=['name', 'version'])['hits']['hits']

In [18]:
print(f'These are the top 5 results of the query "{query}":\n')

for ind, result in enumerate(results):
    length_name = max([len(el['_source']['name']) for el in results]) + 2
    length_version = max([len(el['_source']['version']) for el in results]) + 2

    print(f'\t{str(ind + 1) + ".": <2} {result["_source"]["name"]: <{length_name}} v.{result["_source"]["version"]: <{length_version}} [{str(result["_score"] * 100)[:2]}%]')

These are the top 5 results of the query "driving license registration service":

	1. Vehicle Enquiry API                v.1.1.0   [81%]
	2. Car Registration API               v.1.0.0   [81%]
	3. Motor Vehicle Department, Odisha   v.3.0.0   [78%]
	4. Transport Department, Tamil Nadu   v.3.0.0   [78%]
	5. Transport Department, Haryana      v.3.0.0   [78%]
